In [1]:
from keras.applications import VGG16
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f6384ae2b10> False
<keras.layers.convolutional.Conv2D object at 0x7f62dcd73610> False
<keras.layers.convolutional.Conv2D object at 0x7f62dcd73a50> False
<keras.layers.pooling.MaxPooling2D object at 0x7f62dcd2f050> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc4d2390> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc4e04d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f62dc4e77d0> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc4ecc50> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc4f7b50> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc480290> False
<keras.layers.pooling.MaxPooling2D object at 0x7f62dc48c4d0> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc499610> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc4a44d0> False
<keras.layers.convolutional.Conv2D object at 0x7f62dc4aa910> False
<keras.layers.pooling.MaxPooling2D object at 0x7f62dc4b8b90>

In [4]:
from keras import models
from keras import layers
from keras import optimizers
 
model = models.Sequential()
 
model.add(vgg_conv)
 
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(7, activation='softmax'))
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 3591      
Total params: 40,934,215
Trainable params: 33,298,951
Non-trainable params: 7,635,264
__________________________________

In [5]:
train_dir = '/home/asus/Desktop/Dataset_splitss/train'
validation_dir = '/home/asus/Desktop/Dataset_splitss/validation'

In [6]:
import numpy as np
from random import shuffle

In [7]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
  height_shift_range=0.2,
    horizontal_flip=True,
   fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_batchsize = 124
val_batchsize = 20
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=True)

Found 22047 images belonging to 7 classes.
Found 4131 images belonging to 7 classes.


In [8]:
print(train_generator.class_indices)

{'Paddy': 0, 'corn': 1, 'groundnut': 2, 'pepperbell': 3, 'pomogrnate': 4, 'potato': 5, 'tomato': 6}


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=6,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)


Epoch 1/6
178/177 [==============================] - 3885s 22s/step - loss: 0.3509 - acc: 0.8948 - val_loss: 0.0088 - val_acc: 0.9918
Epoch 2/6
178/177 [==============================] - 3400s 19s/step - loss: 0.0820 - acc: 0.9742 - val_loss: 0.0119 - val_acc: 0.9910
Epoch 3/6
178/177 [==============================] - 3878s 22s/step - loss: 0.0604 - acc: 0.9822 - val_loss: 8.9753e-04 - val_acc: 0.9785
Epoch 4/6
178/177 [==============================] - 3647s 20s/step - loss: 0.0476 - acc: 0.9861 - val_loss: 0.0164 - val_acc: 0.9932
Epoch 5/6
178/177 [==============================] - 4174s 23s/step - loss: 0.0394 - acc: 0.9883 - val_loss: 1.1271e-06 - val_acc: 0.9937
Epoch 6/6
177/177 [============================>.] - ETA: 13s - loss: 0.0324 - acc: 0.9905

In [ ]:
model.save('finalized_typecrop_disease.h5')